# **Projek Akhir Praktikum Data Science**

> Analisis Sentimen Mengenai Vaksin COVID-19 Di Indonesia Menggunakan Metode Naive Bayes Classifier dan NLP Pada Sosial Media Twitter

**Oleh Kelompok 2 :**
1. Hazlan Muhammad Qodri (123190080) @hzlnqodrey
2. Elisia Dwi Rahayu (123190062) @elisiadwirahayu
3. Shania Septika Inayasari (123190055) @shaniainayasari

**Penjelasan Projek :**

Adapun pada penelitian menekankan kepada sentimen masyarakat terhadap mengenai vaksin COVID-19. Proses analisisnya akan dilakukan berdasarkan tweet yang menyertakan tagar vaksin dan pencarian di twitter dengan keyword vaksin covid 19.

## **1. Scraping Data from Twitter**

In [ ]:
%pip install snscrape

In [67]:
import pandas as pd
import numpy as np
import snscrape.modules.twitter as sntwitter
import re


#### Query

In [68]:
# Get All Covid Sentiment Data from January 1st, 2020 until November 1st, 2022
query = "covid since:2020-01-01 until:2022-11-01 lang:id"
limit = 2000 # limit 50k rows

In [69]:
tweets = []

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    else:
        tweets.append([
            tweet.date,
            tweet.username,
            tweet.content
        ])

filename = 'tweets_covid_dataset_2k_raw_noindex.csv'
tweets_df = pd.DataFrame(tweets, columns=['Tanggal', 'Username', 'Text'])
tweets_df.to_csv(filename, index=False)
print('Scraping has completed!')

C:\Users\HAZLAN M QODRI\AppData\Local\Temp\ipykernel_11752\2473716662.py:9: FutureWarning: username is deprecated, use user.username instead
  tweet.username,


Scraping has completed!


## **2. Wrangling Data** (Preprocessing)

In [ ]:
%pip install tweet-preprocessor
%pip install textblob
%pip install wordcloud
%pip install nltk

In [ ]:
import preprocessor as preproc
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import csv

In [58]:
# get data from dataset
data = pd.read_csv('https://raw.githubusercontent.com/hzlnqodrey/projek-akhir-prak-ds-sentimen-analisis-twitter-covid19-nlp-bayes/main/tweets_covid_dataset_50k_raw.csv')

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  50000 non-null  int64 
 1   Tanggal     50000 non-null  object
 2   Username    50000 non-null  object
 3   Text        50000 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


In [60]:
data.sample(n=5)

,Unnamed: 0,Tanggal,Username,Text
26457,26457,2022-10-20 09:29:40+00:00,dinadinda9,"@YanniAffan @alwie888 Busui dulu awal covid tdk boleh divaksin, lah habis itu terbitlah peraturan boleh divaksin 🙃"
1842,1842,2022-10-31 05:09:57+00:00,SZakaera,"@mynohomar Betul. Bantu usahawan kecil, ramai yang terkesan selepas pandemik covid 19."
12234,12234,2022-10-26 09:07:19+00:00,Rini703,@BEMUI_Official Wahai adek adek mahasiswa saya emak2 kelahiran 70 sdh merasakan pemerintahan mbah Harto sampai Jokowi ..baru ini saya merasa Indonesia lebih maju..apalagi setelah covid melanda dunia bersyukur adek adek masih hidup !!!!\n#mahasewa
14095,14095,2022-10-25 09:22:20+00:00,mhdhafizee,"@sillyp1e penat2 tangan ni duk email &amp; komen kat IG Krispy Kreme tu, suruh dia bukak kat JB - way before COVID happen. but they all choose negeri P first. what do do 🙄"
40811,40811,2022-10-13 09:30:00+00:00,sinaksenjaa,Luar biasa sekarang Indonesia bisa memproduksi vaksin Covid-19 sendiri dgn kapasitas kurang lebih 20 jt dosis per tahun \n#JokowiErickVaksinRI https://t.co/Th1iJNcNwB


##### 1. Case Folding

In [61]:
data['Text'] = data['Text'].str.lower()
print('Case Folding Result : \n')
print(data['Text'].head(5))
print(data['Text'].sample(n=5))
print(data['Text'].tail(5))
print('\n\n\n')

Case Folding Result : 

0                         @riderkulineran dulu di barat upn ada warung kayuh bambai. bumbu habangnya sungguh banjar banar. semenjak tahun ke 2 covid mereka tutup. padahal itu warung yang sangat mengobati kerinduan sama masakan banjar. nasi kuning dan soto banjarnya juara sekali 🥺
1                                                                                                                                                                                                                                         segala sakit yg disalahin vaksin covid. lieur.
2    @msaid_didu bukankah kamu dulu bagian rezim?, namun setelah terdepak dan sakit hati karena rezekinya hilang maka jadi sok kritis? dulu mengkritisi vaksin covid, namun saat divaksin pamer di medsos. ucapanmu tidak konsisten bro...semua karena sakit hati atas hilangnya rezeki.
3                                                                                                                                    

In [62]:
data.isnull().sum()

Unnamed: 0    0
Tanggal       0
Username      0
Text          0
dtype: int64

##### 2. Cleaning

In [63]:
def preprocessing_data(x):
    return preproc.clean(x)

data['Text'] = data['Text'].apply(preprocessing_data)
print('Cleaning Result : \n')

Cleaning Result : 



In [64]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', None)
display(data['Text'].head(15))


0                                      dulu di barat upn ada warung kayuh bambai. bumbu habangnya sungguh banjar banar. semenjak tahun ke covid mereka tutup. padahal itu warung yang sangat mengobati kerinduan sama masakan banjar. nasi kuning dan soto banjarnya juara sekali
1                                                                                                                                                                                                                                  segala sakit yg disalahin vaksin covid. lieur.
2         bukankah kamu dulu bagian rezim?, namun setelah terdepak dan sakit hati karena rezekinya hilang maka jadi sok kritis? dulu mengkritisi vaksin covid, namun saat divaksin pamer di medsos. ucapanmu tidak konsisten bro...semua karena sakit hati atas hilangnya rezeki.
3                                                                                                                                                                                 

##### 3. Tokenizing

In [65]:
def tokenize_data(x):
    return word_tokenize(x)

data['Text'] = data['Text'].apply(tokenize_data)
print('Tokenizing Result : \n')


Tokenizing Result : 



In [66]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', None)
display(data['Text'].head(15))

0                                      dulu di barat upn ada warung kayuh bambai. bumbu habangnya sungguh banjar banar. semenjak tahun ke covid mereka tutup. padahal itu warung yang sangat mengobati kerinduan sama masakan banjar. nasi kuning dan soto banjarnya juara sekali
1                                                                                                                                                                                                                                  segala sakit yg disalahin vaksin covid. lieur.
2         bukankah kamu dulu bagian rezim?, namun setelah terdepak dan sakit hati karena rezekinya hilang maka jadi sok kritis? dulu mengkritisi vaksin covid, namun saat divaksin pamer di medsos. ucapanmu tidak konsisten bro...semua karena sakit hati atas hilangnya rezeki.
3                                                                                                                                                                                 

##### 4. Filtering

## **2.1 Stemming**

## **2.2 Menejermah Tweet Clean hasil Preproc ke Bahasa Inggris** (Translating)

## **3. Modelling Data** (Modeling and Training)

## **4. Klasifikasi Data dengan Naive Bayes** (Classification)